## **Mountain Wetland Mapping: Step 2 - classifying high mountain wetlands**

Connect to Google Earth Engine

In [2]:
# import Google Earth Engine API
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-rikebecker')

In [3]:
import geemap #for plotting interactive maps (includes folium)
from ee import batch # for exporting maps/images to google drive
import matplotlib.pyplot as plt # for plotting the historgram
import seaborn as sns # for plotting the historgram
import pandas as pd # for plotting the historgram
import geopandas as gpd
import numpy as np # for further image calculation
import math #for tile processing
import folium
import os
from math import ceil

# **Import data**

In [49]:
# load CSV table from GEE assets
training_data = ee.FeatureCollection('projects/ee-rikebecker/assets/training_data')

In [51]:
# Mountain areas to cut the global images - regions for upscaling
himalaya = ee.Geometry.Rectangle(62, 20, 110, 45)
alps = ee.Geometry.Rectangle(4, 43, 15, 48)
rockyMountains = ee.Geometry.Rectangle(-120, 30, -102, 48)
andes = ee.Geometry.Rectangle(-80, -40, -60, 11)

MountainRegions = ee.FeatureCollection([himalaya, alps, rockyMountains, andes])
MountainRegions_mask = himalaya.union(alps).union(rockyMountains).union(andes)

In [52]:
# import satellite data available in the google earth engine data catalog for the four major mountain regions
ecoregions_1 = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017") # RESOLVE dataset with 846 global ecoregions
dem = ee.Image("USGS/SRTMGL1_003") # Nasa DEM 30m resolution
#flow_accumulation_1 = ee.Image("WWF/HydroSHEDS/15ACC")

In [53]:
# Mountain areas to cut the global images and to use as regions for upscaling
himalaya_dem = dem.clip(himalaya)
himalaya_mask = himalaya_dem.gt(3500)
himalaya_dem_masked = himalaya_dem.updateMask(himalaya_mask)

alps_dem = dem.clip(alps)
alps_mask = alps_dem.gt(1800)
alps_dem_masked = alps_dem.updateMask(alps_mask)

rockyMountains_dem = dem.clip(rockyMountains)
rockyMountains_mask = rockyMountains_dem.gt(2000)
rockyMountains_dem_masked = rockyMountains_dem.updateMask(rockyMountains_mask)

andes_dem = dem.clip(andes)
andes_mask = andes_dem.gt(3500)
andes_dem_masked = andes_dem.updateMask(andes_mask)

# mask for all high mountain regions (DEM > 2000 masl)
dem_clipped = dem.clipToCollection(MountainRegions)
mountain_mask = dem_clipped.gt(2000)

In [54]:
# cut the ecoregions to the mask (move this part also to the loop for training data collection)
def clip_feature(feature):
    return feature.intersection(MountainRegions_mask, ee.ErrorMargin(5))

ecoregions = ecoregions_1.map(clip_feature) #clips to exact extend of MountainRegion mask - use for upscaling.

In [55]:
# clip all DEM derived input images to mountain regions
elevation_temp = dem_clipped.select('elevation') # elevation clipped to MountainRegions extent
slope_temp = ee.Terrain.slope(dem_clipped) # slope clipped to MountainRegions extent
#flow_accumulation_temp = flow_accumulation_1.clip(MountainRegions) # flow accumulation clipped to MountainRegions extent

# **Define functions**

Cloud masking

In [56]:
# write a function for cloud masking
def cloudless(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask_clouds = qa.bitwiseAnd(cloudBitMask).eq(0).And(
      qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask_clouds).divide(10000)#.clip(training_region) #change to '.clip(all_training_regions)' if run for all mountain regions

  # the division by 10,000 is done because Sentinel-2 stores reflectance values as integers scaled by a factor of 10,000.
  # By dividing the pixel values by 10,000, we convert the data back to its original reflectance range (0 to 1).

Function to create indices

In [14]:
# write a function that computes all vegetation indices and adds them as a band
def spectral(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')

  # TCG
  tcg = image.expression(
    '-0.2941*BLUE - 0.243*GREEN - 0.5424*RED + 0.7276*NIR + 0.0713*SWIRI - 0.1608*SWIRII',{
      'BLUE': image.select('B2'),
      'GREEN': image.select('B3'),
      'RED': image.select('B4'),
      'NIR': image.select('B8'),
      'SWIRI': image.select('B11'),
      'SWIRII': image.select('B12'),
    }).rename('tcg')

  # ARI
  ari = image.expression(
    '(B8 / B2) - (B8 / B3)', {
        'B8': image.select(['B8']),
        'B2': image.select(['B2']),
        'B3': image.select(['B3']),
      }).rename('ari')

  # PSRI
  psri = image.expression(
    '(B4 - B2) / B5', {
      'B4': image.select(['B4']),
      'B2': image.select(['B2']),
      'B5': image.select(['B5']),
      }).rename('psri')


  # REIP
  reip = image.expression(
    '702 + 40*((((RED + RE3)/2) - RE1) / (RE2 - RE1))', {
      'RE1': image.select(['B5']),
      'RE2': image.select(['B6']),
      'RE3': image.select(['B7']),
      'RED': image.select(['B4']),
      }).rename('reip')

  # add indices to image collection
  indices = image.addBands(ndvi).addBands(ndwi).addBands(tcg).addBands(ari).addBands(reip).addBands(psri)
  return indices

In [15]:
def clip_image(image):
    return image.clipToCollection(mask)

In [16]:
# mask for angle selection of sentinel-1 data. Selecting images taken with angles between 30-45 degrees.
def mask_ang_gt_30(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.gt(30))

def mask_ang_lt_45(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.lt(45))

In [ ]:
# Define the function to apply the windy day filter.
# def pct_wat(image):
#     # Get the date of the image
#     d = image.date().format('Y-M-d')

#     # Filter the weather data for the given date
#     wx = ee.ImageCollection('NOAA/CFSV2/FOR6H').filterDate(d)

#     # Select the v-component and u-component of wind
#     vWind = wx.select(['v-component_of_wind_height_above_ground'])
#     uWind = wx.select(['u-component_of_wind_height_above_ground'])

#     # Get the maximum wind components
#     a = vWind.max().pow(2)
#     b = uWind.max().pow(2)

#     # Calculate the wind speed
#     ab = a.add(b)
#     ws = ab.sqrt().multiply(3.6)

#     # Update mask based on wind speed less than 12
#     return image.updateMask(ws.lt(12))

In [17]:
# Function to calculate gamma0 and NDPI
def calculate_gamma0_and_ndpi(image):
    # Get the incidence angle in radians
    angle = image.select('angle')
    angle_rad = angle.multiply(math.pi / 180)

    # Calculate gamma0 for VV and VH
    gamma0_vv = image.select('VV').divide(angle_rad.cos()).rename('gamma0_VV')
    gamma0_vh = image.select('VH').divide(angle_rad.cos()).rename('gamma0_VH')

    # Calculate NDPI
    ndpi = gamma0_vv.subtract(gamma0_vh).divide(gamma0_vv.add(gamma0_vh)).rename('NDPI')

    return image.addBands([gamma0_vv, gamma0_vh, ndpi])

In [18]:
# Define the function to apply the convolution filter.
# Define the 3x3 boxcar kernel.
# used to create a circular kernel with a specified radius, unit, and normalization
boxcar = ee.Kernel.circle(radius=3, units='pixels', normalize=True)

def apply_filter(image):
    filtered_image = image.convolve(boxcar)
    return filtered_image

In [19]:
# Define the function to apply the angle correction and convert to gamma0.
def to_gamma0(image):
    # Select the 'angle' band and apply the angle correction
    angle_rad = image.select('angle').multiply(ee.Number(math.pi).divide(180.0))
    cos_angle = angle_rad.cos()
    correction_factor = cos_angle.log10().multiply(10.0)

    # Apply the correction to 'VV' band
    vv_corrected = image.select('VV').subtract(correction_factor)
    vh_corrected = image.select('VH').subtract(correction_factor)

    # Return the image with corrected 'VV' band
    return image.addBands(vv_corrected.rename('VV_gamma0')).addBands(vh_corrected.rename('VH_gamma0'))

In [20]:
# Function to compute the Sigma Lee filter
def sigma_lee(image, kernel_size=3, sigma=0.9):
    # Compute the local mean and variance
    reducer = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.variance(),
        sharedInputs=True
    )

    # Compute the mean and variance for each pixel within the window
    stats = image.reduceNeighborhood(
        reducer=reducer,
        kernel=ee.Kernel.square(kernel_size // 2),
        optimization='window'
    )

    mean = stats.select(0)
    variance = stats.select(1)

    # Compute the noise variance
    noise_variance = variance.sqrt().divide(mean).pow(2).multiply(sigma)

    # Compute the coefficient of variation
    coef_variation = variance.sqrt().divide(mean)

    # Compute the filtered value
    one = ee.Image.constant(1)
    filter_value = one.subtract(noise_variance.divide(variance)).multiply(image.subtract(mean)).add(mean)

    # Mask invalid values
    filter_value = filter_value.updateMask(coef_variation.lte(sigma))

    return filter_value

In [ ]:
# # function for principal component analysis. Check for region!
# def get_covariance_matrix(image, region):
#     # Compute the mean of each band.
#     mean_dict = image.reduceRegion(
#         reducer=ee.Reducer.mean(),
#         geometry=region,
#         scale=10,
#         maxPixels=1e9
#     )
#     means = ee.Image.constant(mean_dict.values(image.bandNames()))

#     # Center the data.
#     centered = image.subtract(means)

#     # Compute the covariance matrix.
#     covar_dict = centered.toArray().reduceRegion(
#         reducer=ee.Reducer.centeredCovariance(),
#         geometry=region,
#         scale=10,
#         maxPixels=1e9
#     )
#     return ee.Array(covar_dict.get('array'))

In [21]:
# Function to extract spectral information at training points
def extract_data(feature):
    return feature.set(input.select(bands).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30  # Spatial resolution in meters
    ))

# **Get Sentinel images and filter by date and cloud cover**

In [22]:
# Rasterize the 'ECO_ID' property into the image
eco_id_image = ecoregions.reduceToImage(
    properties=['ECO_ID'],
    reducer=ee.Reducer.first()
)

# **Prepare training and validation data sets from full training data set**

Separate training and testing datasets

In [57]:
bands = ['ndvi', 'ndwi', 'elevation', 'tpi', 'slope', 'tcg', 'NDPI', 'VV_gamma0', 'VH_gamma0', 'reip', 'ari', 'ecoregion']
target = 'class'

In [24]:
# Filter the training data to remove features with null values in any of the properties
training_data = training_data.filter(ee.Filter.notNull(bands))

training_data = training_data.randomColumn('random')

trainSet = training_data.filter(ee.Filter.lte('random', 0.8)) # 2/3 of data for training //80%
testSet = training_data.filter(ee.Filter.gt('random', 0.8)) # 1/3 of data for validation //20%

# **Train Random Forest Classifier**

In [58]:
# Define the Random Forest classifier
classifier = ee.Classifier.smileRandomForest(50)

# Train the classifier
classifier = classifier.train(
    features=trainSet,  # This should be your training data
    classProperty='class',  # The property containing the class labels
    inputProperties=bands  # The bands to be used as input features
)

In [59]:
# train the probability classifier
# Number of features
numFeatures = len(bands)

# Create and train the smileRandomForest classifier
classifier_prob = ee.Classifier.smileRandomForest(
    numberOfTrees=50,
    variablesPerSplit=int(numFeatures ** 0.5)
).setOutputMode('PROBABILITY')

classifier_prob = classifier_prob.train(trainSet, 'class', bands)

# **Apply classifier to a different year and region of interest**

In [60]:
start_date = '2024-06-01'
end_date = '2024-10-01'

# wet season Peru: october - april ('(YEAR-1)-10-01' - 'YEAR-05-01')
# dry season Peru: may - september ('YEAR-05-01' - 'YEAR-10-01')

In [28]:
Catchment_AGS2 = ee.FeatureCollection('projects/ee-rikebecker/assets/catchment_AGS2')

In [61]:
# set upscale mask
mask_upscale = andes

In [62]:
# sentinel 2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
             .filterBounds(mask_upscale)\
             .filterDate(start_date, end_date)\
             .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))\
             .map(cloudless)
s2_clipped = s2.map(lambda image: image.clip(mask_upscale))
s2_m = s2_clipped.median()

# get sentinel 1 image
s1_pol = ee.ImageCollection('COPERNICUS/S1_GRD') \
           .filterBounds(mask_upscale)\
           .filterDate(start_date, end_date)\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
           .filter(ee.Filter.eq('resolution_meters', 10))
s1_pol = s1_pol.map(lambda image: image.clip(mask_upscale))

In [ ]:
s2.size().getInfo()

In [ ]:
s1_pol.size().getInfo()

In [64]:
# Apply functions to s1
s1_pol1 = s1_pol.map(mask_ang_gt_30)
s1_pol1 = s1_pol1.map(mask_ang_lt_45)
#s1_pol1 = s1_pol1.map(pct_wat)
NDPI = s1_pol1.map(calculate_gamma0_and_ndpi)
NDPI = NDPI.map(apply_filter)
NDPI = NDPI.median()
s1_pol2 = s1_pol.map(to_gamma0)
s1_pol_lee_filtr = s1_pol2.map(sigma_lee)
VV_VH = s1_pol_lee_filtr.mean()
VVsd_VHsd = s1_pol_lee_filtr.reduce(ee.Reducer.stdDev())

# topographic position index (TPI)
focal_mean = elevation_temp.clip(mask_upscale).focalMean(5)
tpi = elevation_temp.clip(mask_upscale).subtract(focal_mean)
tpi = tpi.rename(['tpi'])

# clip flow accumulation, slope and elevation
#flow_accumulation = flow_accumulation_temp.clip(mask_upscale)
elevation = elevation_temp.clip(mask_upscale)
slope = slope_temp.clip(mask_upscale)

# Compute TWI: ln(A/tan(beta)) with slope in radians
# slope_radians = slope.multiply(ee.Number(math.pi).divide(180))
# twi = flow_accumulation.divide(slope_radians.tan()).log()
# twi = twi.rename(['twi'])
# validDataImage = twi.selfMask()
# filledImage = twi.unmask().focal_mean(radius=3, kernelType='circle', iterations=1)
# twi = validDataImage.unmask(filledImage) # twi image without no-data gaps

# sentinel 2 indices
spec_indices = spectral(s2_m)

# ecoregions
ecoregion = eco_id_image.clip(mask_upscale)
ecoregion = ecoregion.rename(['ecoregion'])

# # calculate PC1 from bands 2,3,4 and 8
# bands_req = s2_m.select(['B2', 'B3', 'B4', 'B8'])
# # Calculate the covariance matrix of the selected bands.
# covariance_matrix = get_covariance_matrix(bands_req, mask_upscale) # make sure to change if mask changes
# # Perform eigen analysis (PCA).
# eig = covariance_matrix.eigen()
# # Extract eigenvalues and eigenvectors.
# eigenvalues = eig.slice(1, 0, 1)
# eigenvectors = eig.slice(1, 1)

# # Convert the bands to an array image.
# array_image = bands_req.toArray()
# # Compute the principal components.
# principal_components = ee.Image(eigenvectors).matrixMultiply(array_image.toArray(1)).arrayProject([0]).arrayFlatten([['pc1', 'pc2', 'pc3', 'pc4']])
# # Select the first principal component (PC1).
# pc1 = principal_components.select('pc1')
# pc1 = pc1.rename(['pc1'])

indices = spec_indices.addBands(slope).addBands(elevation).addBands(tpi).addBands(VV_VH).addBands(VVsd_VHsd).addBands(NDPI).addBands(ecoregion)#.addBands(pc1)

bands = ['ndvi', 'ndwi', 'elevation', 'tpi', 'slope', 'tcg', 'NDPI', 'VV_gamma0', 'VH_gamma0', 'reip', 'ari', 'ecoregion']#, 'pc1']
input_yoi = indices.select(bands)

# **Classify year of interest**

# **Get probabilities and export highly probable wetlands (>90%)**

In [65]:
classified = input_yoi.classify(classifier, 'classification')

In [66]:
# Filter samples for each class
wetlands = classified.updateMask(classified.eq(1))
no_wetlands = classified.updateMask(classified.eq(0))

In [75]:
# apply probability classifier
classified_prob = input_yoi.classify(classifier_prob)

# Define the class of interest and probability threshold
class_name = 'classification'
probability = classified_prob.select(class_name)# Extract probability for the specific class
high_prob = probability.gt(0.90)# Threshold the probability to keep only high-probability areas

# Mask the original image using the high probability threshold
high_prob_image = wetlands.updateMask(high_prob)
#high_prob_image = classified_prob

In [69]:
final_prob_image = classified_prob.unitScale(0, 1).multiply(255).toByte()

# **Export results to drive**

In [89]:
#export to drive
region = andes
image = high_prob_image

def create_grid(geometry, max_dimension):
    """Divide the geometry into smaller tiles."""
    # Get the coordinates of the region.
    if isinstance(geometry, ee.FeatureCollection):
        geometry = geometry.geometry()

    bounds = geometry.bounds().coordinates().get(0).getInfo()
    lon_min, lat_min = bounds[0][0], bounds[0][1]
    lon_max, lat_max = bounds[2][0], bounds[2][1]

    # Calculate the number of tiles needed, always round up.
    lon_diff = lon_max - lon_min
    lat_diff = lat_max - lat_min

    num_lon_tiles = int(ceil(lon_diff / max_dimension))
    num_lat_tiles = int(ceil(lat_diff / max_dimension))

    # Create the tiles.
    tiles = []
    for i in range(num_lon_tiles):
        for j in range(num_lat_tiles):
            tile = ee.Geometry.Rectangle([
                lon_min + i * max_dimension,
                lat_min + j * max_dimension,
                min(lon_min + (i + 1) * max_dimension, lon_max),
                min(lat_min + (j + 1) * max_dimension, lat_max)
            ])
            tiles.append(tile)
    return tiles

# Create tiles with a maximum dimension of X degrees.
tiles = create_grid(region, max_dimension=2)

In [ ]:
len(tiles)

In [ ]:
# Export each tile
if isinstance(region, ee.FeatureCollection):
    region = region.geometry()

for i, tile in enumerate(tiles):
    # Ensure the intersection geometry is valid
    intersection = region.intersection(tile, ee.ErrorMargin(1))

    # Clip the image to the intersection geometry
    clipped_tile = image.clip(intersection)

    # Get the bounds of the tile for export
    tile_bounds = tile.bounds().getInfo()['coordinates']

    # Example: export the clipped tile to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=clipped_tile,
        description=f'Wetlands_2024_dry_andes_{i}',
        scale=30,
        region=tile_bounds,
        fileFormat = 'GeoTIFF',
        folder='earth_engine_exports'
    )
    task.start()

    print(f'Started export task for tile {i}')

    # Optional: Monitor the task
    #import time
    #while task.active():
     #  print(f'Processing tile {i}')
     #  time.sleep(300)
   # print('Task status:', task.status())

# **Validation of classified images**

In [ ]:

# Classify the validation dataset and calculate the confusion matrix
confusion_matrix = classified_prob.errorMatrix('class', 'classification')

# Get the number of classes from the confusion matrix
num_classes = confusion_matrix.array().length().get([0])

# Function to create a feature for each row of the confusion matrix
def create_feature(actual_class):
    row = confusion_matrix.array().slice(0, actual_class, actual_class.add(1)).project([1])
    row_dict = ee.Dictionary.fromLists(
        ee.List.sequence(0, num_classes.subtract(1)),
        row.toList()
    )
    return ee.Feature(None, row_dict.set('actual', actual_class))

# Create a feature collection by mapping over the number of classes
matrix_fc = ee.FeatureCollection(
    ee.List.sequence(0, num_classes.subtract(1)).map(create_feature)
)

# Export the confusion matrix to Google Drive as a CSV file
export_task = ee.batch.Export.table.toDrive(
    collection=matrix_fc,
    description='ConfusionMatrixExport',
    fileFormat='CSV',
    folder='earth_engine_exports'
)

# Start the export task
export_task.start()